In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Concatenate,Dense,Input,Embedding,Bidirectional,LSTM,GRU
import keras.backend as K
from tensorflow.keras.utils import to_categorical

In [4]:
imdb_df = pd.read_csv("/data2/scratch/rohitr/datasets/IMDB_Dataset.csv")

In [5]:
d = {'negative':0,'positive':1}

In [6]:
imdb_df['sentiment'] = imdb_df['sentiment'].apply(lambda x : d.get(x))

In [7]:
X_train,X_test,y_train,y_test = train_test_split(imdb_df['review'],imdb_df['sentiment'],test_size=0.2)

In [8]:
X_train = X_train.to_numpy()

In [9]:
X_test = X_test.to_numpy()

In [10]:
y_train = y_train.to_numpy()

In [11]:
y_test = y_test.to_numpy()

In [12]:
#clean_text
import re
def clean_text(text,punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']):
    
    
    text = re.sub(r'https?://\S+|www\.\S+', '',text)
    text = re.sub(r'<.*?>', '',text)
    # Removing the punctuations
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")
            
    text = text.lower()
    # Removing stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    
    
    text = re.sub(r'\s+', ' ',text).strip()
    return text

In [14]:
X_train = [clean_text(x) for x in X_train]

In [15]:
X_test = [clean_text(x) for x in X_test]

In [16]:
len(X_test)

10000

In [17]:
len(X_train)

40000

In [18]:
t=Tokenizer()
t.fit_on_texts(X_train)
text_matrix=t.texts_to_sequences(X_train)

In [19]:
test_matrix = t.texts_to_sequences(X_test)

In [20]:
vocab_length = len(t.word_index) + 1

In [21]:
vocab_length

192305

In [22]:
from keras.preprocessing.sequence import pad_sequences
text_pad = pad_sequences(text_matrix, maxlen=200, padding='post')

In [23]:
test_pad = pad_sequences(test_matrix, maxlen=200, padding='post')

In [24]:
from keras.layers import Layer
import keras.backend as K

In [25]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [26]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [28]:
vocab_size = 30000  # Only consider the top 30k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)


17473536/17464789 [==============================] - 3s 0us/step


KeyboardInterrupt: 

In [27]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(200,))
embedding_layer = TokenAndPositionEmbedding(200, vocab_length, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1,activation='sigmoid',trainable = True)(x)

model = keras.Model(inputs=inputs, outputs=outputs)


2022-09-02 19:24:29.303644: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-02 19:24:32.214060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38418 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [28]:
y_train

array([0, 0, 1, ..., 0, 1, 1])

In [29]:
hist = model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(
    text_pad, y_train, batch_size=32, epochs=2, validation_split= 0.2
)


2022-09-02 19:24:36.847288: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
   7/1000 [..............................] - ETA: 21s - loss: 0.6989 - accuracy: 0.5580   

2022-09-02 19:24:46.630879: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1000/1000 [==============================] - 33s 23ms/step - loss: 0.3645 - accuracy: 0.8312 - val_loss: 0.2953 - val_accuracy: 0.8761
Epoch 2/2
1000/1000 [==============================] - 22s 22ms/step - loss: 0.1538 - accuracy: 0.9443 - val_loss: 0.2841 - val_accuracy: 0.8851


In [30]:
model.evaluate(test_pad,y_test)

313/313 [==============================] - 5s 13ms/step - loss: 0.2924 - accuracy: 0.8866


[0.29236993193626404, 0.8866000175476074]